## 201710773 박영제

In [150]:
import findspark
findspark.init()
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

# 문제 1 시작

# 문제 1-1의 답

In [151]:
import os
from pyspark.sql import Row
df = spark.read.csv("data/서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv", encoding='euc-kr', header='True')

In [152]:
df.printSchema()

root
 |-- 사용월: string (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: string (nullable = true)
 |-- 무임승차인원: string (nullable = true)
 |-- 유임하차인원: string (nullable = true)
 |-- 무임하차인원: string (nullable = true)
 |-- 작업일자: string (nullable = true)



#### 인원에 관련된 스키마의 내용들은 integer형으로 바뀌는 것이 옳기에 변환합니다.

In [153]:
from pyspark.sql.types import IntegerType

df = df.withColumn('유임승차인원', df['유임승차인원'].cast("integer"))\
        .withColumn('무임승차인원', df['무임승차인원'].cast("integer"))\
        .withColumn('유임하차인원', df['유임하차인원'].cast("integer"))\
        .withColumn('무임하차인원', df['무임하차인원'].cast("integer"))

In [154]:
df.printSchema()

root
 |-- 사용월: string (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: integer (nullable = true)
 |-- 무임승차인원: integer (nullable = true)
 |-- 유임하차인원: integer (nullable = true)
 |-- 무임하차인원: integer (nullable = true)
 |-- 작업일자: string (nullable = true)



# 문제 1-2의 답

In [155]:
print("데이터의 건수:",df.count(),'개')

데이터의 건수: 55171 개


# 문제 1-3의 답

In [156]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

win = Window.partitionBy("호선명").orderBy(F.col("유임승차인원").desc())

In [157]:
from pyspark.sql.functions import rank

rankDf = df.withColumn("rank", rank().over(win))
rankDf.show(30)

+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|
|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|
|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|
|201703|일산선|    화정|      499911|      141327|      529944|      145502|20170403|   4|
|201510|일산선|    화정|      499388|      520595|      140545|      144148|20151108|   5|
|201905|일산선|    화정|      499218|      149790|      520966|      152915|20190603|   6|
|201612|일산선|    화정|      497514|      132807|      521734|      136086|20170108|   7|
|201805|일산선|    화정|      494994|      139239|      515748|      142997|20180603|   8|
|201610|일산선|    화정|      492146|      137462|      513849|      140303|201611

# 문제 1-4의 답

In [158]:
rankDf.filter(rankDf['rank'] == 1).show(30)

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|   1|
|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|   1|
|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|   1|
|201905|    우이신설선|        북한산보국문|      161867|       45041|      141742|       45814|20190603|   1|
|201512|        분당선|                야탑|      769442|      157846|      809197|      156034|20160108|   1|
|201512|         7호선|      가산디지털단지|     1245731|       82588|     1243464|       78424|20160108|   1|
|201905|        수인선|              인하대|      200474|       28246|      189885|   

# 문제 1-5의 답

In [159]:
from pyspark.sql.window import Window

byLineAvg = Window.partitionBy('호선명')
rankDf = rankDf.withColumn("meanByLine", F.avg(rankDf['유임승차인원']).over(byLineAvg))

In [160]:
from pyspark.sql.window import Window

byLineStd = Window.partitionBy('호선명')
rankDf = rankDf.withColumn("sdByLine", F.stddev(rankDf['유임승차인원']).over(byLineStd))

In [161]:
rankDf = rankDf.withColumn("zscore", (F.col('유임승차인원')-F.col('meanByLine'))/F.col('sdByLine'))

In [162]:
rankDf.select('호선명','유임승차인원','meanByLine','sdByLine','zscore').show()

+------+------------+------------------+------------------+--------------------+
|호선명|유임승차인원|        meanByLine|          sdByLine|              zscore|
+------+------------+------------------+------------------+--------------------+
|일산선|      237714|231630.21716649432|127730.48519577553| 0.04762984203951724|
|일산선|      363144|231630.21716649432|127730.48519577553|  1.0296193788971475|
|일산선|      294566|231630.21716649432|127730.48519577553|  0.4927232738295993|
|일산선|      208122|231630.21716649432|127730.48519577553|-0.18404546988499038|
|일산선|      237249|231630.21716649432|127730.48519577553|0.043989364206153625|
|일산선|      159355|231630.21716649432|127730.48519577553| -0.5658415612820729|
|일산선|           7|231630.21716649432|127730.48519577553|  -1.813374597391374|
|일산선|      171040|231630.21716649432|127730.48519577553|-0.47435987637270977|
|일산선|      147292|231630.21716649432|127730.48519577553| -0.6602826023656541|
|일산선|      432339|231630.21716649432|127730.48519577553|  1.5713

# 문제 2 시작

In [163]:
from sklearn.datasets import make_regression
X, y, coef = make_regression(n_samples = 200,
                            n_features = 4,
                            n_informative = 3,
                            n_targets = 1,
                            noise = 0.0,
                            coef = True)

# 문제 2-1의 답

In [164]:
def gradientDescent(x,y,theta,alpha,numIterations):
    for i in range(numIterations):
        h = np.dot(x,theta)
        error = h-y
        cost = np.sum((h-y)**2)/len(x)
        gradient = np.dot(x.T,error)*2/len(x)
        theta -= alpha*gradient

    return theta

In [165]:
import numpy as np

alpha = 0.01
theta = np.array(np.ones([X.shape[1]]))

In [166]:
w4,w3,w2,w1 = gradientDescent(X, y, theta, alpha, 700)

##### gradient방법에서 도출된 계수와 거의 일치한다.

In [231]:
print("예측:",[w4,w3,w2,w1])
print("coef:",coef)

예측: [23.418167691851977, 38.87142850570739, 90.57174195549767, -3.3659664431461196e-05]
coef: [23.41820526 38.87122219 90.57197391  0.        ]


# 문제 2-2의 답

In [168]:
import pandas as pd

pdDf = pd.DataFrame(X)
pdDf['y'] = y

blobDf = spark.createDataFrame(pdDf)

In [169]:
blobDf = blobDf.withColumnRenamed("0", "x0")\
            .withColumnRenamed("1", "x1")\
            .withColumnRenamed("2", "x2")\
            .withColumnRenamed("3", "x3")

In [170]:
blobDf.show(10)

+-------------------+-------------------+--------------------+--------------------+--------------------+
|                 x0|                 x1|                  x2|                  x3|                   y|
+-------------------+-------------------+--------------------+--------------------+--------------------+
| 0.2753910465802733|-0.8487638481871769|  -0.557176224337695| -0.3194524092805922|  -77.00787452397248|
|-0.4658141225995953| 0.5329100256157533|-0.10820307490851391|  2.6149614406286954|0.006167200099902459|
|  1.803468753210042|  -1.15035394953932|  0.8238924292175069| -1.0761875535398011|   72.13990106867794|
|-1.0172023083449973| 2.1470439547868327| 0.24514767635212772|0.030462558894281795|   81.84067911303453|
|0.40062162494063036|-2.0473291598843635|  -0.331745609630397| 0.43782995658782975|  -100.2472019185588|
|-0.7191901671560752| 1.8486926681409195|  1.0934616552337013| -0.8600628180408383|  154.05578101035226|
|-1.6977719297243352| 1.3509634860123205| -0.3028378475

# 문제 2-3의 답

In [171]:
(trainDf, testDf) = blobDf.randomSplit([0.6, 0.4])

# 문제 2-4의 답

In [172]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols = ["x0","x1","x2","x3"],
                                   outputCol = 'features')

In [173]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='y')

In [174]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[va,lr])

In [175]:
model = pipeline.fit(trainDf)
modelTrainDf = model.transform(trainDf)

In [176]:
print("Coefficients: {}".format(model.stages[-1].coefficients))
print("Intercept: {:.3f}".format(model.stages[-1].intercept))

Coefficients: [23.418205255034245,38.87122218622474,90.5719739058417,-2.828151260420349e-15]
Intercept: 0.000


In [177]:
modelTrainDf.select('y','prediction').show(10)

+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
|  7.046728285491227|  7.046728285491198|
|  -27.9954281589948|-27.995428158994816|
|-177.76934902525045|-177.76934902525045|
| 100.97338403058319| 100.97338403058316|
|-127.46347435194868|-127.46347435194868|
|  9.105928376291153|  9.105928376291137|
| -141.6916514438945| -141.6916514438945|
| 2.1285910974660993|  2.128591097466094|
|-161.38637560475334| -161.3863756047533|
| 17.714584312858655| 17.714584312858644|
+-------------------+-------------------+
only showing top 10 rows



# 문제 2-5의 답

In [178]:
modelTestDf = model.transform(testDf)

In [179]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")

In [180]:
print("r2:",evaluator.evaluate(modelTestDf))

r2: 1.0


# 문제 3 시작

# 문제 3-1 답

##### 파일 작성 시, 긍정은 1, 부정은 0으로 하여 레이블링 했습니다.

In [227]:
import os
spiderDf = spark\
        .read\
        .options(header='false', inferschema='true', delimiter='\t')\
        .csv(os.path.join('data', 'spiderman.txt'))

In [228]:
spiderDf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)



In [229]:
spiderDf = spiderDf.withColumnRenamed("_c0", "review")\
                    .withColumnRenamed("_c1", "label")

In [230]:
spiderDf.show(10)

+----------------------------------+-----+
|                            review|label|
+----------------------------------+-----+
|고구마 먹은 영화 유치해서 못보겠다|  0.0|
|  노잼이네요.. 왜 배우들이 출연...|  0.0|
|   제발 뻘짓 좀 그만하셨으면......|  0.0|
|         스토리가 매우 매우 아쉽다|  0.0|
|             영화 약간 이해가 안됨|  0.0|
|좋아하는 배우들인데 최근 영화라...|  0.0|
| 그지같네 도대체 이 따위 영화를...|  0.0|
|  스크린보다 시계를 더 많이 봄....|  0.0|
| 이때까지 본 영화중에 제일 최악...|  0.0|
|    나만 짜증.. 줄거리 한줄평.....|  0.0|
+----------------------------------+-----+
only showing top 10 rows



# 문제 3-2의 답

In [185]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="review", outputCol="tokens")
tokDf = tokenizer.transform(spiderDf)

#### 불필요 단어 제거

In [186]:
def RemoveUselessWords(wordList):
    removed=list()
    
    for w in wordList:
        removed.append(w.replace("!","").replace(".","").replace("..","").replace('...',"").replace('....',"").replace(",","").replace('ㅋ',"").replace('ㄹㅇ',"").replace(' ',"").replace('ㅠ',""))
                       
    return removed

In [187]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType, StringType

removeUdf=f.udf(RemoveUselessWords, ArrayType(StringType()))

In [188]:
wordsDf = tokDf.withColumn('words', removeUdf(f.col('tokens')))

In [189]:
wordsDf.show(5)

+----------------------------------+-----+------------------------------+------------------------------+
|                            review|label|                        tokens|                         words|
+----------------------------------+-----+------------------------------+------------------------------+
|고구마 먹은 영화 유치해서 못보겠다|  0.0|[고구마, 먹은, 영화, 유치해...|[고구마, 먹은, 영화, 유치해...|
|  노잼이네요.. 왜 배우들이 출연...|  0.0|[노잼이네요.., 왜, 배우들이...|[노잼이네요, 왜, 배우들이, ...|
|   제발 뻘짓 좀 그만하셨으면......|  0.0|[제발, 뻘짓, 좀, 그만하셨으...|[제발, 뻘짓, 좀, 그만하셨으...|
|         스토리가 매우 매우 아쉽다|  0.0|[스토리가, 매우, 매우, 아쉽다]|[스토리가, 매우, 매우, 아쉽다]|
|             영화 약간 이해가 안됨|  0.0|    [영화, 약간, 이해가, 안됨]|    [영화, 약간, 이해가, 안됨]|
+----------------------------------+-----+------------------------------+------------------------------+
only showing top 5 rows



#### 불용어 제거

In [190]:
stopwords = ['그', '수', '본', '등', '때', '걸', '다', '게', '날', '짱', '거', '중', '더', '넘', '두', '달', '올', '뻔', '단', '개', '듯']

In [191]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="words", outputCol="nostops")

In [192]:
stop.setStopWords(stopwords)

StopWordsRemover_1afe127dddcd

In [193]:
stopDf=stop.transform(wordsDf)
stopDf.select('review','nostops').show(5, truncate=False)

+---------------------------------------------------+--------------------------------------------------------+
|review                                             |nostops                                                 |
+---------------------------------------------------+--------------------------------------------------------+
|고구마 먹은 영화 유치해서 못보겠다                 |[고구마, 먹은, 영화, 유치해서, 못보겠다]                |
|노잼이네요.. 왜 배우들이 출연했는지 이해가 잘 안됨 |[노잼이네요, 왜, 배우들이, 출연했는지, 이해가, 잘, 안됨]|
|제발 뻘짓 좀 그만하셨으면.... 이건 여러모로 낭비다.|[제발, 뻘짓, 좀, 그만하셨으면, 이건, 여러모로, 낭비다]  |
|스토리가 매우 매우 아쉽다                          |[스토리가, 매우, 매우, 아쉽다]                          |
|영화 약간 이해가 안됨                              |[영화, 약간, 이해가, 안됨]                              |
+---------------------------------------------------+--------------------------------------------------------+
only showing top 5 rows



# 문제 3-3의 답

In [194]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")

In [195]:
hashDf = hashTF.transform(stopDf)

In [196]:
idf = IDF(inputCol="hash", outputCol="idf")

In [197]:
idfModel = idf.fit(hashDf)

In [198]:
idfDf = idfModel.transform(hashDf)

In [218]:
idfDf.select('review','idf').show(5)

+----------------------------------+--------------------+
|                            review|                 idf|
+----------------------------------+--------------------+
|고구마 먹은 영화 유치해서 못보겠다|(262144,[114067,1...|
|  노잼이네요.. 왜 배우들이 출연...|(262144,[14123,77...|
|   제발 뻘짓 좀 그만하셨으면......|(262144,[24759,45...|
|         스토리가 매우 매우 아쉽다|(262144,[8121,118...|
|             영화 약간 이해가 안됨|(262144,[35471,10...|
+----------------------------------+--------------------+
only showing top 5 rows



# 문제 3-4의 답

In [200]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(featuresCol='hash', labelCol='label', modelType='multinomial', predictionCol='prediction',smoothing=1.0)

In [201]:
model=nb.fit(hashDf)
predictions=model.transform(hashDf)

In [202]:
predictions.select('review','label','prediction').show(10)

+----------------------------------+-----+----------+
|                            review|label|prediction|
+----------------------------------+-----+----------+
|고구마 먹은 영화 유치해서 못보겠다|  0.0|       0.0|
|  노잼이네요.. 왜 배우들이 출연...|  0.0|       0.0|
|   제발 뻘짓 좀 그만하셨으면......|  0.0|       0.0|
|         스토리가 매우 매우 아쉽다|  0.0|       0.0|
|             영화 약간 이해가 안됨|  0.0|       0.0|
|좋아하는 배우들인데 최근 영화라...|  0.0|       0.0|
| 그지같네 도대체 이 따위 영화를...|  0.0|       0.0|
|  스크린보다 시계를 더 많이 봄....|  0.0|       0.0|
| 이때까지 본 영화중에 제일 최악...|  0.0|       0.0|
|    나만 짜증.. 줄거리 한줄평.....|  0.0|       0.0|
+----------------------------------+-----+----------+
only showing top 10 rows



# 문제 3-5의 답

In [203]:
(trainDf, testDf) = hashDf.randomSplit([0.5, 0.5])

In [204]:
splitModel = nb.fit(trainDf)

#### test data에 대한 예측

In [205]:
testPredict = splitModel.transform(testDf)

In [206]:
testPredict.select('review','label','prediction').show(15)

+----------------------------------+-----+----------+
|                            review|label|prediction|
+----------------------------------+-----+----------+
|        감동 그 자체 스파이더맨 짱|  1.0|       0.0|
|고구마 먹은 영화 유치해서 못보겠다|  0.0|       0.0|
| 그냥 대박임 그 시절 스파이더맨...|  1.0|       0.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|  1.0|       1.0|
| 그지같네 도대체 이 따위 영화를...|  0.0|       0.0|
|  극장에서 보는 거 적극 추천 최고!|  1.0|       1.0|
| 다음 날 새벽 출근임에도 불구하...|  1.0|       0.0|
| 똥 싸다가 막힌 느낌 안보는거를...|  0.0|       0.0|
|                띵작입니다 ㄹㅇ!!!|  1.0|       0.0|
|로맨틱 코미디로 방향을 잡았으면...|  0.0|       0.0|
|  보는 내내 명치끝이 탁 막혀서 ...|  1.0|       1.0|
|  스토리 개 말도 안되는 억지 고...|  0.0|       0.0|
| 어릴적 영화관에서 처음으로 본 ...|  1.0|       0.0|
| 어설픈 연출력 어설픈 결말 현실...|  0.0|       0.0|
|  어우 핵노잼 영화가 왜 이렇게 ...|  0.0|       0.0|
+----------------------------------+-----+----------+
only showing top 15 rows



In [207]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [208]:
evaluator.evaluate(testPredict)

0.7307692307692308

# 문제 3-6의 답

In [209]:
testDf = spark.createDataFrame(
    [
        ["고구가 먹은 느낌 킬링타음으로도 아쉽다"],
        ["내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다"],
        ["보다가 채널 돌림 재미없다"],
        ["최고의 영화"],
        ["여운이 긴 명작"],
        ["띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요"]
    ],
    ['sentences']
)

#### hashingTF 계산

In [210]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="sentences", outputCol="tokens")
test_tokDf = tokenizer.transform(testDf)

In [211]:
test_wordsDf = test_tokDf.withColumn('words', removeUdf(f.col('tokens')))

In [212]:
test_hashTF = HashingTF(inputCol="nostops", outputCol="hash")

In [213]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[stop,test_hashTF])

In [214]:
test_model = pipeline.fit(test_wordsDf)
test_df_fin = test_model.transform(test_wordsDf)

In [215]:
test_df_fin.show()

+----------------------------------+-------------------------------+-------------------------------+-------------------------------+--------------------+
|                         sentences|                         tokens|                          words|                        nostops|                hash|
+----------------------------------+-------------------------------+-------------------------------+-------------------------------+--------------------+
|고구가 먹은 느낌 킬링타음으로도...| [고구가, 먹은, 느낌, 킬링타...| [고구가, 먹은, 느낌, 킬링타...| [고구가, 먹은, 느낌, 킬링타...|(262144,[8121,745...|
| 내돈 아깝 핵노잼 영화가 이렇게...| [내돈, 아깝, 핵노잼, 영화가...| [내돈, 아깝, 핵노잼, 영화가...| [내돈, 아깝, 핵노잼, 영화가...|(262144,[35595,47...|
|         보다가 채널 돌림 재미없다| [보다가, 채널, 돌림, 재미없다]| [보다가, 채널, 돌림, 재미없다]| [보다가, 채널, 돌림, 재미없다]|(262144,[103639,2...|
|                       최고의 영화|                 [최고의, 영화]|                 [최고의, 영화]|                 [최고의, 영화]|(262144,[17393,14...|
|                    여운이 긴 명작|             [여운이, 긴, 명작]|             [여운이

#### 예측

In [216]:
test_predictions = model.transform(test_df_fin)

In [217]:
test_predictions.select('sentences','prediction').show(truncate=False)

+------------------------------------------------+----------+
|sentences                                       |prediction|
+------------------------------------------------+----------+
|고구가 먹은 느낌 킬링타음으로도 아쉽다          |0.0       |
|내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다|0.0       |
|보다가 채널 돌림 재미없다                       |0.0       |
|최고의 영화                                     |1.0       |
|여운이 긴 명작                                  |1.0       |
|띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요|1.0       |
+------------------------------------------------+----------+

